In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load data & setup pandas
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv", index_col='id'
#                     , nrows=500000
                   )
test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv", index_col='id'
#                     , nrows=250000
                   )
print(train.shape)
print(test.shape)

train.shape (1000000, 286) \
test.shape  (500000, 285)

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import gc

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                    
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
gc.collect()

In [ ]:
#train.info(verbose=True, show_counts=True)
# Only float16 and int8 columns are present in our data set

In [ ]:
# y = train["target"].copy()
# train.drop(columns="target", inplace=True)

In [ ]:
bin_cols = list(train.select_dtypes('int8').columns)
bin_cols.remove('target')
cont_cols = list(train.select_dtypes('float16').columns)

In [ ]:
train.target.hist()

Target set is not skewed

In [ ]:
train[cont_cols].astype(float).describe()

Continious features already scaled to the [0;1] interval

In [ ]:
train.iloc[:, :12].hist(bins=100, figsize = (15,20))

Continious features have a different nature.
There are features with left/right tails.
Some features with multimodal distribution.

In [ ]:
sns.set(rc={"figure.figsize":(15, 10)})
sns.boxplot(data=train.iloc[:, :12])


In [ ]:
train[bin_cols].astype('category').describe()

In [ ]:
corr = train[['target'] + cont_cols].corr()
corr.style.background_gradient(cmap='coolwarm').format(precision = 3)

In [ ]:
corr = train[['target'] + bin_cols].corr()
corr.style.background_gradient(cmap='coolwarm').format(precision = 3)

f22 has a highest correlation with target

In [ ]:
#Save data in binary format
train.reset_index(inplace=True)
train.to_feather('train.feather')
test.reset_index(inplace=True)
test.to_feather('test.feather')
